<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Text-Classification" data-toc-modified-id="Text-Classification-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Text Classification</a></span><ul class="toc-item"><li><span><a href="#Subjectivity-Dataset" data-toc-modified-id="Subjectivity-Dataset-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Subjectivity Dataset</a></span></li><li><span><a href="#Tokenization" data-toc-modified-id="Tokenization-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Tokenization</a></span><ul class="toc-item"><li><span><a href="#Simple-Tokenization" data-toc-modified-id="Simple-Tokenization-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Simple Tokenization</a></span></li><li><span><a href="#Much-better-tokenization-with-Spacy" data-toc-modified-id="Much-better-tokenization-with-Spacy-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Much better tokenization with Spacy</a></span></li></ul></li><li><span><a href="#Split-dataset-in-train-and-validation" data-toc-modified-id="Split-dataset-in-train-and-validation-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Split dataset in train and validation</a></span></li><li><span><a href="#Word-to-index-mapping" data-toc-modified-id="Word-to-index-mapping-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Word to index mapping</a></span></li><li><span><a href="#Sentence-encoding" data-toc-modified-id="Sentence-encoding-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Sentence encoding</a></span></li><li><span><a href="#Embedding-layer" data-toc-modified-id="Embedding-layer-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Embedding layer</a></span></li><li><span><a href="#Continuous-Bag-of-Words-Model" data-toc-modified-id="Continuous-Bag-of-Words-Model-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Continuous Bag of Words Model</a></span></li></ul></li><li><span><a href="#Training-the-CBOW-model" data-toc-modified-id="Training-the-CBOW-model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Training the CBOW model</a></span></li><li><span><a href="#Data-loaders-for-SGD" data-toc-modified-id="Data-loaders-for-SGD-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data loaders for SGD</a></span></li></ul></div>

In [1]:
# import pytorch libraries
%matplotlib inline
import torch 
import torch.autograd as autograd 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

# Text Classification
In this part of the tutorial we develop a continuous bag of words (CBOW) model for a text classification task described [here]( https://people.cs.umass.edu/~miyyer/pubs/2015_acl_dan.pdf). The CBOW model was first described [here](https://arxiv.org/pdf/1301.3781.pdf)

## Subjectivity Dataset
The subjectivity dataset has 5000 subjective and 5000 objective processed sentences. To get the data:
```
wget http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
```

In [2]:
def unpack_dataset():
    ! wget http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
    ! mkdir data
    ! tar -xvf rotten_imdb.tar.gz -C data

In [3]:
unpack_dataset()

--2020-02-19 11:49:47--  http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.20
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.20|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 519599 (507K) [application/x-gzip]
Saving to: ‘rotten_imdb.tar.gz’

rotten_imdb.tar.gz  100%[===================>] 507.42K   558KB/s    in 0.9s    

2020-02-19 11:49:49 (558 KB/s) - ‘rotten_imdb.tar.gz’ saved [519599/519599]

x quote.tok.gt9.5000
x plot.tok.gt9.5000
x subjdata.README.1.0


In [4]:
!ls data

plot.tok.gt9.5000   quote.tok.gt9.5000  subjdata.README.1.0


In [5]:
! head -2 data/plot.tok.gt9.5000

the movie begins in the past where a young boy named sam attempts to save celebi from a hunter . 
emerging from the human psyche and showing characteristics of abstract expressionism , minimalism and russian constructivism , graffiti removal has secured its place in the history of modern art while being created by artists who are unconscious of their artistic achievements . 


In [6]:
from pathlib import Path
PATH = Path("data")
list(PATH.iterdir())

[PosixPath('data/plot.tok.gt9.5000'),
 PosixPath('data/subjdata.README.1.0'),
 PosixPath('data/quote.tok.gt9.5000')]

## Tokenization
Tokenization is the task of chopping up text into pieces, called tokens.

spaCy is an open-source software library for advanced Natural Language Processing. Here we will use it for tokenization.  

### Simple Tokenization

In [7]:
# We need each line in the file 
def read_file(path):
    """ Read file returns a list of lines.
    """
    with open(path, encoding = "ISO-8859-1") as f:
        content = f.readlines()
    return content

In [8]:
obj_lines = read_file(PATH/"plot.tok.gt9.5000")

In [9]:
obj_lines[0]

'the movie begins in the past where a young boy named sam attempts to save celebi from a hunter . \n'

In [10]:
np.array(obj_lines[0].strip().lower().split(" "))

array(['the', 'movie', 'begins', 'in', 'the', 'past', 'where', 'a',
       'young', 'boy', 'named', 'sam', 'attempts', 'to', 'save', 'celebi',
       'from', 'a', 'hunter', '.'], dtype='<U8')

### Much better tokenization with Spacy

In [11]:
#!pip install -U spacy

In [12]:
import spacy

In [13]:
# first time run this
#!python3 -m spacy download en

In [14]:
tok = spacy.load('en')

In [15]:
obj_lines = read_file(PATH/"plot.tok.gt9.5000")

In [16]:
len(obj_lines)

5000

In [17]:
obj_lines[0]

'the movie begins in the past where a young boy named sam attempts to save celebi from a hunter . \n'

In [18]:
test = tok(obj_lines[0])

In [19]:
np.array([x for x in test])

array([the, movie, begins, in, the, past, where, a, young, boy, named,
       sam, attempts, to, save, celebi, from, a, hunter, ., 
], dtype=object)

## Split dataset in train and validation

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
sub_content = read_file(PATH/"quote.tok.gt9.5000")
obj_content = read_file(PATH/"plot.tok.gt9.5000")
sub_content = np.array([line.strip().lower() for line in sub_content])
obj_content = np.array([line.strip().lower() for line in obj_content])
sub_y = np.zeros(len(sub_content))
obj_y = np.ones(len(obj_content))
X = np.append(sub_content, obj_content)
y = np.append(sub_y, obj_y)

In [22]:
X[0], y[0]

('smart and alert , thirteen conversations about one thing is a small gem .',
 0.0)

In [23]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
X_train[:5], y_train[:5]

(array(['will god let her fall or give her a new path ?',
        "the director's twitchy sketchbook style and adroit perspective shifts grow wearisome amid leaden pacing and indifferent craftsmanship ( most notably wretched sound design ) .",
        "welles groupie/scholar peter bogdanovich took a long time to do it , but he's finally provided his own broadside at publishing giant william randolph hearst .",
        'based on the 1997 john king novel of the same name with a rather odd synopsis : " a first novel about a seasoned chelsea football club hooligan who represents a disaffected society operating by brutal rules .',
        'yet , beneath an upbeat appearance , she is struggling desperately with the emotional and physical scars left by the attack .'],
       dtype='<U691'), array([1., 0., 0., 1., 1.]))

## Word to index mapping
In interest of time we will tokenize without spaCy. Here we will compute a vocabulary of words based on the training set and a mapping from word to an index.

In [25]:
from collections import defaultdict

In [26]:
def get_vocab(content):
    """Computes Dict of counts of words.
    
    Computes the number of times a word is on a document.
    """
    vocab = defaultdict(float)
    for line in content:
        words = set(line.split())
        for word in words:
            vocab[word] += 1
    return vocab      

In [27]:
#Getting the vocabulary from the training set
word_count = get_vocab(X_train)

In [28]:
#word_count

In [29]:
len(word_count.keys())

21415

In [30]:
# let's delete words that are very infrequent
for word in list(word_count):
    if word_count[word] < 5:
        del word_count[word]

In [31]:
len(word_count.keys())

4065

In [32]:
## Finally we need an index for each word in the vocab
vocab2index = {"<PAD>":0, "UNK":1} # init with padding and unknown
words = ["<PAD>", "UNK"]
for word in word_count:
    vocab2index[word] = len(words)
    words.append(word)

In [33]:
#vocab2index

## Sentence encoding
Here we encode each sentence as a sequence of indices corresponding to each word.

In [34]:
x_train_len = np.array([len(x.split()) for x in X_train])
x_val_len = np.array([len(x.split()) for x in X_val])

In [35]:
np.percentile(x_train_len, 95) # let set the max sequence len to N=40

43.0

In [36]:
X_train[0]

'will god let her fall or give her a new path ?'

In [37]:
# returns the index of the word or the index of "UNK" otherwise
vocab2index.get("?", vocab2index["UNK"])

7

In [38]:
np.array([vocab2index.get(w, vocab2index["UNK"]) for w in X_train[0].split()])

array([ 4,  3,  5, 11, 12,  8,  9, 11, 10,  2,  6,  7])

In [39]:
def encode_sentence(s, N=40):
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in s.split()])
    l = min(N, len(enc1))
    enc[:l] = enc1[:l]
    return enc

In [40]:
encode_sentence(X_train[0])

array([ 4,  3,  5, 11, 12,  8,  9, 11, 10,  2,  6,  7,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0], dtype=int32)

In [41]:
x_train_len = np.minimum(x_train_len, 40)
x_val_len = np.minimum(x_val_len, 40)

In [42]:
x_train = np.vstack([encode_sentence(x) for x in X_train])
x_train.shape

(8000, 40)

In [43]:
x_val = np.vstack([encode_sentence(x) for x in X_val])
x_val.shape

(2000, 40)

## Embedding layer
Most deep learning models use a dense vectors of real numbers as representation of words (word embeddings), as opposed to a one-hot encoding representations. The module torch.nn.Embedding is used to represent word embeddings. It takes two arguments: the vocabulary size, and the dimensionality of the embeddings. The embeddings are initialized with random vectors. 

In [44]:
# an Embedding module containing 10 words with embedding size 4
# embedding will be initialized at random
embed = nn.Embedding(10, 4, padding_idx=0)
embed.weight

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [-2.0121,  0.5527, -1.2455,  0.2041],
        [ 0.3683,  0.5744, -0.8790, -1.4613],
        [ 0.2536, -0.3544,  1.6831,  0.7288],
        [ 0.6326, -0.5219, -1.7885, -0.3624],
        [ 0.8093,  0.2436, -0.2002, -0.1828],
        [ 0.4526, -2.4755,  0.9809, -0.0230],
        [-0.4837, -0.4338, -0.2915,  0.6802],
        [-0.2113, -0.2717, -0.0352,  0.1727],
        [ 0.3391, -0.2622,  0.5261,  0.1645]], requires_grad=True)

Note that the `padding_idx` has embedding vector 0.

In [45]:
# given a list of ids we can "look up" the embedding corresponing to each id
# can you see that some vectors are the same?
a = torch.LongTensor([[1,4,1,5,1,0]])
embed(a)

tensor([[[-2.0121,  0.5527, -1.2455,  0.2041],
         [ 0.6326, -0.5219, -1.7885, -0.3624],
         [-2.0121,  0.5527, -1.2455,  0.2041],
         [ 0.8093,  0.2436, -0.2002, -0.1828],
         [-2.0121,  0.5527, -1.2455,  0.2041],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]], grad_fn=<EmbeddingBackward>)

This would be the representation of a sentence with words with indices [1,4,1,5,1] and a padding at the end. Bellow we have an example in which we have two sentences. the first sentence has length 3 and the last sentence has length 2. In order to use a tensor we use padding at the end of the second sentence. 

In [46]:
a = torch.LongTensor([[1,4,1], [1,3,0]])

Our model takes an average of the word embedding of each word. Here is how we do it.

In [47]:
s = torch.FloatTensor([3, 2]) # here is the size of the vector

In [48]:
embed(a)

tensor([[[-2.0121,  0.5527, -1.2455,  0.2041],
         [ 0.6326, -0.5219, -1.7885, -0.3624],
         [-2.0121,  0.5527, -1.2455,  0.2041]],

        [[-2.0121,  0.5527, -1.2455,  0.2041],
         [ 0.2536, -0.3544,  1.6831,  0.7288],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]], grad_fn=<EmbeddingBackward>)

In [49]:
embed(a).sum(dim=1)

tensor([[-3.3916,  0.5835, -4.2794,  0.0458],
        [-1.7585,  0.1983,  0.4377,  0.9329]], grad_fn=<SumBackward1>)

In [50]:
sum_embs = embed(a).sum(dim=1) 
sum_embs/ s.view(s.shape[0], 1)

tensor([[-1.1305,  0.1945, -1.4265,  0.0153],
        [-0.8793,  0.0992,  0.2188,  0.4665]], grad_fn=<DivBackward0>)

## Continuous Bag of Words Model

In [51]:
class CBOW(nn.Module):
    def __init__(self, vocab_size, emb_size=100):
        super(CBOW, self).__init__()
        self.word_emb = nn.Embedding(vocab_size, emb_size, padding_idx=0)
        self.linear = nn.Linear(emb_size, 1)
        
    def forward(self, x, s):
        x = self.word_emb(x)
        x = x.sum(dim=1)/ s
        x = self.linear(x)
        return x

In [52]:
model = CBOW(vocab_size=5, emb_size=3)

In [53]:
model.word_emb.weight

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000],
        [ 0.7458, -1.2917,  0.5418],
        [ 0.7172, -0.3131, -1.4046],
        [-1.0790, -0.3249,  1.0058],
        [ 1.6530, -0.9145,  0.6278]], requires_grad=True)

In [54]:
s = s.view(s.shape[0], 1)
model(a, s)

tensor([[-0.7422],
        [-1.0013]], grad_fn=<AddmmBackward>)

# Training the CBOW model 

In [55]:
V = len(words)
model = CBOW(vocab_size=V, emb_size=50)
print(V)

4067


In [56]:
def val_metrics(model):
    model.eval()
    x = torch.LongTensor(x_val) #.cuda()
    y = torch.Tensor(y_val).unsqueeze(1) #).cuda()
    s = torch.Tensor(x_val_len).view(x_val_len.shape[0], 1)
    y_hat = model(x, s)
    loss = F.binary_cross_entropy_with_logits(y_hat, y)
    y_pred = y_hat > 0
    correct = (y_pred.float() == y).float().sum()
    accuracy = correct/y_pred.shape[0]
    return loss.item(), accuracy.item()

In [57]:
# accuracy of a random model should be around 0.5
val_metrics(model)

(0.6952854990959167, 0.49549999833106995)

In [58]:
def train_epocs(model, epochs=10, lr=0.01):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for i in range(epochs):
        model.train()
        x = torch.LongTensor(x_train)  #.cuda()
        y = torch.Tensor(y_train).unsqueeze(1)
        s = torch.Tensor(x_train_len).view(x_train_len.shape[0], 1)
        y_hat = model(x, s)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        val_loss, val_accuracy = val_metrics(model)
        print("train_loss %.3f val_loss %.3f val_accuracy %.3f" % (loss.item(), val_loss, val_accuracy))

In [59]:
train_epocs(model, epochs=10, lr=0.1)

train_loss 0.694 val_loss 0.668 val_accuracy 0.547
train_loss 0.658 val_loss 0.576 val_accuracy 0.779
train_loss 0.564 val_loss 0.486 val_accuracy 0.833
train_loss 0.463 val_loss 0.388 val_accuracy 0.872
train_loss 0.354 val_loss 0.325 val_accuracy 0.877
train_loss 0.278 val_loss 0.273 val_accuracy 0.892
train_loss 0.217 val_loss 0.254 val_accuracy 0.901
train_loss 0.185 val_loss 0.247 val_accuracy 0.904
train_loss 0.159 val_loss 0.247 val_accuracy 0.905
train_loss 0.137 val_loss 0.259 val_accuracy 0.906


In [60]:
train_epocs(model, epochs=10, lr=0.01)

train_loss 0.122 val_loss 0.255 val_accuracy 0.909
train_loss 0.117 val_loss 0.254 val_accuracy 0.910
train_loss 0.114 val_loss 0.253 val_accuracy 0.910
train_loss 0.112 val_loss 0.253 val_accuracy 0.911
train_loss 0.109 val_loss 0.253 val_accuracy 0.910
train_loss 0.106 val_loss 0.253 val_accuracy 0.911
train_loss 0.103 val_loss 0.253 val_accuracy 0.912
train_loss 0.100 val_loss 0.254 val_accuracy 0.911
train_loss 0.097 val_loss 0.255 val_accuracy 0.910
train_loss 0.094 val_loss 0.256 val_accuracy 0.910


# Data loaders for SGD

Nearly all of deep learning is powered by one very important algorithm: **stochastic gradient descent (SGD)**. SGD can be seeing as an approximation of **gradient descent** (GD). In GD you have to run through *all* the samples in your training set to do a single itaration. In SGD you use *only one* or *a subset*  of training samples to do the update for a parameter in a particular iteration. The subset use in every iteration is called a **batch** or **minibatch**.

In [61]:
from torch.utils.data import Dataset, DataLoader

Next we are going to create a data loader. The data loader provides the following features:
* Batching the data
* Shuffling the data
* Load the data in parallel using multiprocessing workers.

In [62]:
def encode_sentence2(s, N=40):
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in s.split()])
    l = min(N, len(enc1))
    enc[:l] = enc1[:l]
    return enc, l

In [63]:
encode_sentence2(X_train[0])

(array([ 4,  3,  5, 11, 12,  8,  9, 11, 10,  2,  6,  7,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0], dtype=int32), 12)

In [64]:
class SubjectivityDataset(Dataset):
    def __init__(self, X, y):
        self.x = X
        self.y = y
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x = self.x[idx]
        x, s = encode_sentence2(x)
        return x, self.y[idx], s
    
sub_dataset_train = SubjectivityDataset(X_train, y_train)

In [65]:
train_loader = DataLoader(sub_dataset_train, batch_size=5, shuffle=True)
x, y, s = next(iter(train_loader))

In [66]:
x, y, s

(tensor([[ 199,   90,   14,   23,    1, 3433,   51, 3724,    1,  804,  420,   13,
           274,   20,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0],
         [   1, 1111,   10,  974, 2948, 3109,  154,   28, 1381,  188,  363,   28,
           496, 1098,  171, 1763,   28,  121,   10, 3102,  171,   34, 1158, 2542,
           101,  460,   20,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0],
         [   1, 3955,  706,  160,   28,   73, 1730,   42, 2654,   14, 3190,    1,
            51,    1,   13,    1,   81,   14,  162,   20,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0],
         [  14,  118,   74,  874,    1,    1,   13,  401,   42,  298, 1241,   20,
            29,   96, 2611,  130, 2196,    1,  209,   14, 1475,   20,    0,

In [67]:
model = CBOW(vocab_size=V, emb_size=50)

In [68]:
train_loader = DataLoader(sub_dataset_train, batch_size=500, shuffle=True)

In [69]:
def train_epocs(model, epochs=10, lr=0.01):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for i in range(epochs):
        total_loss = 0
        total = 0
        model.train()
        for x, y, s in train_loader:
            x = x.type(torch.LongTensor)  #.cuda()
            y = y.type(torch.FloatTensor).unsqueeze(1)
            s = s.type(torch.Tensor).view(s.shape[0], 1)
            y_hat = model(x, s)
            loss = F.binary_cross_entropy_with_logits(y_hat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += x.size(0)*loss.item()
            total += x.size(0)
        train_loss = total_loss/total
        val_loss, val_accuracy = val_metrics(model)
        
        print("train_loss %.3f val_loss %.3f val_accuracy %.3f" % (train_loss, val_loss, val_accuracy))

In [70]:
train_epocs(model, epochs=10)

train_loss 0.653 val_loss 0.598 val_accuracy 0.772
train_loss 0.511 val_loss 0.436 val_accuracy 0.840
train_loss 0.343 val_loss 0.317 val_accuracy 0.877
train_loss 0.240 val_loss 0.265 val_accuracy 0.899
train_loss 0.186 val_loss 0.242 val_accuracy 0.905
train_loss 0.151 val_loss 0.234 val_accuracy 0.906
train_loss 0.127 val_loss 0.233 val_accuracy 0.910
train_loss 0.109 val_loss 0.234 val_accuracy 0.910
train_loss 0.094 val_loss 0.239 val_accuracy 0.908
train_loss 0.081 val_loss 0.247 val_accuracy 0.905
